<a href="https://colab.research.google.com/github/Quillbolt/colabnotebook/blob/main/retrainingarcface6_11_1230h.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install facenet-pytorch

In [ ]:

batch_size = 32
epochs = 8
workers = 0 if os.name == 'nt' else 8

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training

In [ ]:
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [ ]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [ ]:
data_dir = '/content/drive/My Drive/godeyedata/train'
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))



In [ ]:
traindir = '/content/drive/My Drive/godeyedata/train'
valdir = '/content/drive/My Drive/godeyedata/val'

In [ ]:
train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
          ]))

NameError: ignored

In [ ]:
loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

for i, (x, y) in enumerate(loader):
    mtcnn(x, save_path=y)
    print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')

TypeError: ignored

In [ ]:
val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ])))

NameError: ignored

In [ ]:
model  = torch.load('/content/drive/My Drive/Face-Recognition/work_space/model/model_ir_se50.pth')

UnpicklingError: ignored

In [ ]:

resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

In [ ]:
trans = transforms.Compose([
    np.float32,
    transforms.ToTensor()
    
])
dataset = datasets.ImageFolder(data_dir ,  transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds))

In [ ]:
def adjust_learning_rate(optimizer, gamma, epoch, step_index, iteration, epoch_size):
    """Sets the learning rate
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    warmup_epoch = -1
    if epoch <= warmup_epoch:
        lr = 1e-6 + (initial_lr-1e-6) * iteration / (epoch_size * warmup_epoch)
    else:
        lr = initial_lr * (gamma ** (step_index))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

In [ ]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])


NameError: ignored

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy}

NameError: ignored

In [ ]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------
Valid |     1/1    | loss:    1.9211 | fps:    0.4721 | acc:    0.0769   

Epoch 1/8
----------
Train |     2/2    | loss:    1.5732 | fps:   64.8894 | acc:    0.3577   
Valid |     1/1    | loss:    0.6259 | fps:   23.2615 | acc:    0.7692   

Epoch 2/8
----------
Train |     2/2    | loss:    0.1320 | fps:   96.0022 | acc:    1.0000   
Valid |     1/1    | loss:   13.0053 | fps:   23.7348 | acc:    0.3077   

Epoch 3/8
----------
Train |     2/2    | loss:    0.0278 | fps:   96.5136 | acc:    1.0000   
Valid |     1/1    | loss:    5.6494 | fps:   23.3826 | acc:    0.3846   

Epoch 4/8
----------
Train |     2/2    | loss:    0.0090 | fps:   96.4181 | acc:    1.0000   
Valid |     1/1    | loss:    0.8263 | fps:   23.4406 | acc:    0.7692   

Epoch 5/8
----------
Train |     2/2    | loss:    0.0086 | fps:   96.2859 | acc:    1.0000   
Valid |     1/1    | loss:    0.7685 | fps:   23.8987 | acc:    0.7692   

Epoch 6/8
----------
Train |     2/2    | loss:    0.0

In [ ]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('/content/drive/My Drive/godeyedata/test')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

In [ ]:
resnet=torch.load('/content/drive/My Drive/Pytorch_Retinaface/weights/Resnet50_Final.pth')

In [ ]:

aligned = []
names = []
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('Face detected with probability: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

Face detected with probability: 0.999999
Face detected with probability: 0.999987
Face detected with probability: 0.999994
Face detected with probability: 0.999954
Face detected with probability: 0.999987
Face detected with probability: 0.999975
Face detected with probability: 0.997339
Face detected with probability: 0.999648
Face detected with probability: 0.999966
Face detected with probability: 0.999921
Face detected with probability: 0.999996
Face detected with probability: 0.999995
Face detected with probability: 1.000000
Face detected with probability: 1.000000
Face detected with probability: 1.000000
Face detected with probability: 1.000000
Face detected with probability: 0.999994


In [ ]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

In [ ]:
import pandas as pd


In [ ]:
dists = [[(e1 - e2).norm().item() for e2 in embeddings] for e1 in embeddings]
print(pd.DataFrame(dists, columns=names, index=names))

                                     Huỳnh Tuấn Minh - MT-014  ...  Trần Tiến Hòa-MT-002
Huỳnh Tuấn Minh - MT-014                             0.000000  ...                   2.754488
Huỳnh Tuấn Minh - MT-014                             2.513082  ...                   2.957537
Huỳnh Tuấn Minh - MT-014                             1.811291  ...                   3.006946
Huỳnh Tuấn Minh - MT-014                             1.522675  ...                   2.806876
Huỳnh Tuấn Minh - MT-014                             2.772112  ...                   3.619390
MT - 009 Đỗ Văn Trường - done                     1.787390  ...                   2.199817
MT - 009 Đỗ Văn Trường - done                     2.923374  ...                   2.325509
MT - 009 Đỗ Văn Trường - done                     1.925269  ...                   2.393872
Nguyễn Duy Khánh-MT-013                              2.162966  ...                   2.920080
Nguyễn Hoàng Đức - MT-006